In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
import time
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from matplotlib import pyplot
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import plotly.express as px
from lightgbm import LGBMRegressor
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Bu Notebook Data analizi, görselleştirmesi ve bazı Eleştiriler içermektedir.

In [ ]:
train = pd.read_csv("../input/datathon2022/train.csv")
test = pd.read_csv("../input/datathon2022/testFeatures.csv")

In [ ]:
print("train shape row length:", train.shape[0])
print("train data shape column number:", train.shape[1])

In [ ]:
train['tarih'] = pd.to_datetime(train['tarih'])
train["Yıl"] = train["tarih"].dt.year
train["Ay"] = train["tarih"].dt.month
train["ürün besin değeri"] = train["ürün besin değeri"].astype(int)
train["ürün fiyatı"] = train["ürün fiyatı"].astype(float)
train['kriz'] = (train['tarih'] > "2018-07-01").astype("int")
train['kriz2'] = (train['tarih'] > "2020-04-01").astype("int")
train['pandemi'] = (train['tarih'] > "2020-02-01").astype("int")

train.info()

In [ ]:
# Datamızda eksik değer (missing value) var mı kontrol edelim ?
tablo_0_1 =  pd.DataFrame(train.isna().sum())
tablo_0_1.style.background_gradient(cmap = "turbo")

# Yukarıda oluşturulan kolonlar oluşturacağımız model için oldukça önemlidir. Yıl ve ay değişkeleri ilerideki fiyat tahminleri için belirleyici olacaktır. Türkiye için gıda fiyatları her yıl, her ay hatta bazı durumlarda her gün artmaktadır.

#yıllara göre değişen ürün kategorisi fiyatları.

In [ ]:
tablo_0 = pd.DataFrame(train["ürün kategorisi"].value_counts())
tablo_0.style.background_gradient(cmap = "turbo")

In [ ]:
tablo_1 = pd.DataFrame(train.groupby(["Yıl","ürün kategorisi"])["ürün fiyatı"].mean())
tablo_1.style.background_gradient(cmap = "turbo")

# Yukarıdaki tabloda basit bir groupby operasyonu ile yıllara göre ürün kategorilerinin ortalama fiyat değişimleri gösterilmiştir.
# Ürün kategorilerine göre fiyatların artış oranı;

In [ ]:
print("Et kategorisindeki ürünlerin artış oranı: %{:.3f}".format((55.776927/20.122269)*100))
print("Kuruyemiş kategorisindeki ürünlerin artış oranı: %{:.3f}".format((46.771347/17.109031)*100))
print("Süt ürünleri ve kahvaltılık kategorisindeki ürünlerin artış oranı: %{:.3f}".format((39.914133/14.503876)*100))
print("Tahıl ve ürünleri kategorisindeki ürünlerin artış oranı: %{:.3f}".format((14.837746/5.414829)*100))
print("Sebze ve bakliyat kategorisindeki ürünlerin artış oranı: %{:.3f}".format((23.208409/8.454689)*100))
print("Meyve kategorisindeki ürünlerin artış oranı: %{:.3f}".format((15.571872/5.662894)*100))

In [ ]:
ürünler = ["et", "kuruyemiş", "meyve", "sebze ve bakliyat", "süt ürünleri ve kahvaltılık", "tahıl ve ürünleri"]
Artış_oranları = [2.772*100,2.734*100,2.750*100,2.745*100,2.752*100,2.740*100]

plt.style.use("ggplot")
fig = plt.figure(figsize =(22, 10))
plt.bar(ürünler, Artış_oranları, color="#4CAD39",
        edgecolor = "#1C4814")
plt.ylim([260,280])  #grafiğin daha okunur olması adına ylim methodu ile grafiğimizi daha okunur hale getirdik.
plt.xticks(rotation=40, color="#0F0F34", size=12)
plt.yticks(rotation=35, color="#0F0F34", size=10)
plt.title("Farklı ürün kategorilerine sahip ürünlerin 2016-2020 arasındaki fiyat artış oranı (% cinsinden)", color = "#902009",
          size=18)

In [ ]:
temp = train[["ürün besin değeri","ürün fiyatı"]].describe().T
temp.style.background_gradient(cmap = "turbo")

# Describe tablosunda "ürün besin değerleri" ve "fiyat" kolonunda negatif veya herhangi şüphe uyandırıcı bir değer gözükmemektedir.

In [ ]:
train["ürün"].value_counts()
#Her üründen 2880 adet bulunmaktadır ve 79 adet ürün var.

In [ ]:
tablo_6 = pd.DataFrame(train["ürün kategorisi"].value_counts())
tablo_6.style.background_gradient(cmap = "turbo")

In [ ]:
fig = px.pie(train, names='ürün kategorisi', title='Ürün Kategorileri Yüzdeliklerinin Pie Grafiği Üzerinde Gösterimi')
fig.show()

In [ ]:
plt.figure(figsize=(18,10))
train["market"].value_counts().sort_values().plot(kind = 'bar')
plt.title("Datada bulunan 'market' değişkenleri eşit sayıdadır")

In [ ]:
train["şehir"].value_counts()

In [ ]:
train.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
train["ürün"] = labelencoder.fit_transform(train['ürün'])

train = train.set_index("tarih")

In [ ]:
test['tarih'] = pd.to_datetime(test['tarih'])
test["Yıl"] = test["tarih"].dt.year
test["Ay"] = test["tarih"].dt.month
test["ürün besin değeri"] = test["ürün besin değeri"].astype(int)
test['kriz'] = (test['tarih'] > "2018-07-01").astype("int")
test['kriz2'] = (test['tarih'] > "2020-04-01").astype("int")
test['pandemi'] = (test['tarih'] > "2020-02-01").astype("int")

test["ürün"] = labelencoder.fit_transform(test['ürün'])
test = test.set_index("tarih")

train = train.drop(["market", "ürün üretim yeri"], axis=1)
test = test.drop(["market", "ürün üretim yeri"], axis=1)

In [ ]:
X = train.drop(["ürün fiyatı"], axis=1)
y = train["ürün fiyatı"]

In [ ]:
pd.set_option('display.max_columns', 30)
train.head()

In [ ]:
import catboost as cb

categorical_features = ['ürün kategorisi', 'şehir']
train_dataset = cb.Pool(X, y, cat_features=categorical_features)

In [ ]:
model = cb.CatBoostRegressor(loss_function='RMSE', task_type='GPU')

model.fit(train_dataset)

In [ ]:
import matplotlib.pyplot as plt

sorted_feature_importance = model.feature_importances_.argsort()
plt.barh(X.columns[sorted_feature_importance], 
        model.feature_importances_[sorted_feature_importance], 
        color='turquoise')
plt.xlabel("Catboost Feature Importance")

In [ ]:
vals = [0.3, 0.3, 0.35, 0.4, 0.40, 0.45, 0.45, 0.5, 0.5, 0.5, 0.55, 0.55]

multiplier = test.reset_index()[['tarih']]
dates = ["2021-01-01", "2021-02-01", "2021-03-01", "2021-04-01", "2021-05-01", 
      "2021-06-01", "2021-07-01", "2021-08-01", "2021-09-01", "2021-10-01", 
      "2021-11-01", "2021-12-01"]
multiplier

In [ ]:
mults = pd.DataFrame({"tarih": dates, "multipliers": vals})
mults["tarih"] = pd.to_datetime(mults.tarih)
mults.set_index("tarih", inplace=True)
multiplier = multiplier.join(mults, on=["tarih"], how="left")

In [ ]:
submission_dataset = cb.Pool(test.drop(["id"], axis=1), cat_features=categorical_features)

pred = model.predict(submission_dataset)
pred = pred + pred * multiplier.multipliers

In [ ]:
submission = pd.DataFrame(columns=["id", "ürün fiyatı"])
submission["ürün fiyatı"] = pred
submission["id"] = test.id.values

submission.to_csv("submission.csv", index=False)